In [1]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns



In [2]:
data = pd.read_csv("anto_of_anto_data.csv")

FileNotFoundError: [Errno 2] File b'anto_of_anto_data.csv' does not exist: b'anto_of_anto_data.csv'

In [18]:
data.head()

,Unnamed: 0,Sentence,anto_Sentence-1,anto_Sentence-2,anto_Sentence-3
0,2926,A federal magistrate in Fort Lauderdale ordere...,A federal magistrate in Fort Lauderdale disarr...,A federal magistrate in Fort Lauderdale deregu...,A unitary magistrate in Fort Lauderdale disarr...
1,298,""" No one has anything to fear from being corre...",""" all one has anything to fear from being corr...",""" No one has anything to fearlessness from bei...",""" yes one has anything to fearlessness from be..."
2,3160,""" I have always tried to be honest with you an...",""" I have always tried to be dishonest with you...",""" I have never tried to be dishonest with you ...",""" I have never tried to be dishonest with you ..."
3,1496,The greenback scored beefy gains against the e...,The greenback scored beefy gains against the e...,The greenback scored beefy gains against the e...,The greenback scored beefy reduce against the ...
4,720,The best the investigators can do is nitpick a...,The best the investigators can do is nitpick a...,The worst the investigators can do is nitpick ...,The ill the investigators can do is nitpick ab...


In [19]:
sent= data.Sentence
sent1 = data["anto_Sentence-1"]
sent2 = data["anto_Sentence-2"]
sent3 = data["anto_Sentence-3"]


In [20]:
def cos_sim(a, b):
	"""Takes 2 vectors a, b and returns the cosine similarity according 
	to the definition of the dot product
	"""
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)

#USE

In [21]:
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [22]:
# # positive data
# embed_p1 = embed(data.sentence1.sample(1000))
# embed_p2 = embed(data.sentence2.sample(1000))

# random sample data
embed_sent = embed(sent)
embed_p1 = embed(sent1)
embed_p2 = embed(sent2)
embed_p3 = embed(sent3)

In [23]:
embed_p1.shape

TensorShape([3500, 512])

In [24]:
data["Sent-Syn_Sent-1_Sim"] = [cos_sim(embed_sent[i],embed_p1[i]) for i in range(len(embed_sent))]
data["Sent-Syn_Sent-2_Sim"] = [cos_sim(embed_sent[i],embed_p2[i]) for i in range(len(embed_sent))]
data["Sent-Syn_Sent-3_Sim"] = [cos_sim(embed_sent[i],embed_p3[i]) for i in range(len(embed_sent))]

data["Sent-Syn_Sent-1_Sim"].mean(),data["Sent-Syn_Sent-2_Sim"].mean(),data["Sent-Syn_Sent-3_Sim"].mean()

(0.9537076354026794, 0.9147304892539978, 0.8857790231704712)

In [25]:
with open("jumbled_results.txt","w") as f:
  f.write("USE \n")
  f.write("n=1 %f" % data["Sent-Syn_Sent-1_Sim"].mean())
  f.write("\n")
  f.write("n=2 %f" % data["Sent-Syn_Sent-2_Sim"].mean())
  f.write("\n")
  f.write("n=3 %f" % data["Sent-Syn_Sent-3_Sim"].mean())
  f.write("\n")
  f.write("\n")

# SentBERT

In [26]:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 4.0 MB/s 
     |████████████████████████████████| 4.0 MB 10.6 MB/s 
     |████████████████████████████████| 1.2 MB 47.9 MB/s 
     |████████████████████████████████| 77 kB 8.1 MB/s 
     |████████████████████████████████| 895 kB 46.9 MB/s 
     |████████████████████████████████| 6.6 MB 48.8 MB/s 
     |████████████████████████████████| 596 kB 59.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=a18305706cde244943df5ec331d53ce9ef3bcd05f5c749fee7c27d87c0e717e7
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer
model_sbert = SentenceTransformer('nli-distilroberta-base-v2')


In [28]:
sbert_sent_embed = [model_sbert.encode(i) for i in sent]
sbert_sent1_embed = [model_sbert.encode(i) for i in sent1]
sbert_sent2_embed = [model_sbert.encode(i) for i in sent2]
sbert_sent3_embed = [model_sbert.encode(i) for i in sent3]

In [30]:
sent_sent1 = [cos_sim(sbert_sent_embed[i],sbert_sent1_embed[i]) for i in range(len(sbert_sent_embed))]
sent_sent2 = [cos_sim(sbert_sent_embed[i],sbert_sent2_embed[i]) for i in range(len(sbert_sent_embed))]
sent_sent3 = [cos_sim(sbert_sent_embed[i],sbert_sent3_embed[i]) for i in range(len(sbert_sent_embed))]

np.mean(sent_sent1),np.mean(sent_sent2),np.mean(sent_sent3)

(0.943612, 0.8986837, 0.868949)

In [31]:
with open("jumbled_results.txt","a") as f:
  f.write("Sbert \n")
  f.write("n=1 %f" % np.mean(sent_sent1))
  f.write("\n")
  f.write("n=2 %f" % np.mean(sent_sent2))
  f.write("\n")
  f.write("n=3 %f" % np.mean(sent_sent3))
  f.write("\n")
  f.write("\n")

# LASER

In [32]:
!pip install laserembeddings

     |████████████████████████████████| 859 kB 5.0 MB/s 
     |████████████████████████████████| 45 kB 4.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=3822e8d42cee61acbaf96c9162f0630a4443a3c4a62d0be75b212e38d8132bcd
  Stored in directory: /root/.cache/pip/wheels/d1/ff/0e/e00ff1e22100702ac8b24e709551ae0fb29db9ffc843510a64
Successfully built sacremoses
  Attempting uninstall: sacremoses
    Found existing installation: sacremoses 0.0.49
    Uninstalling sacremoses-0.0.49:
      Successfully uninstalled sacremoses-0.0.49


In [33]:
from laserembeddings import Laser


In [34]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [35]:
laser = Laser()

In [36]:
laser_embed_sent = laser.embed_sentences(sent,lang='en')
laser_embed_sent1 = laser.embed_sentences(sent1,lang='en')
laser_embed_sent2 = laser.embed_sentences(sent2,lang='en')
laser_embed_sent3 = laser.embed_sentences(sent3,lang='en')


In [37]:
laser_sent_sent1 = [cos_sim(laser_embed_sent[i],laser_embed_sent1[i]) for i in range(len(laser_embed_sent))]
laser_sent_sent2 = [cos_sim(laser_embed_sent[i],laser_embed_sent2[i]) for i in range(len(laser_embed_sent))]
laser_sent_sent3 = [cos_sim(laser_embed_sent[i],laser_embed_sent3[i]) for i in range(len(laser_embed_sent))]

np.mean(laser_sent_sent1),np.mean(laser_sent_sent2),np.mean(laser_sent_sent3)

(0.9780925, 0.9605844, 0.9490205)

In [38]:
with open("jumbled_results.txt","a") as f:
  f.write("LASER \n")
  f.write("n=1 %f" % np.mean(laser_sent_sent1))
  f.write("\n")
  f.write("n=2 %f" % np.mean(laser_sent_sent2))
  f.write("\n")
  f.write("n=3 %f" % np.mean(laser_sent_sent3))
  f.write("\n")
  f.write("\n")

# DOC2VEC

In [ ]:
from random import randint

import numpy as np
import torch

In [ ]:
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')


In [41]:
tagged_sent = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sent)]
tagged_sent1 = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sent1)]
tagged_sent2 = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sent2)]
tagged_sent3 = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sent3)]



In [42]:
model_sent = Doc2Vec(vector_size=20,window=5,min_count=1,workers=2,dm=1,epochs=20)
model_sent.build_vocab(tagged_sent)

model_sent1 = Doc2Vec(vector_size=20,window=5,min_count=1,workers=2,dm=1,epochs=20)
model_sent1.build_vocab(tagged_sent1)

model_sent2 = Doc2Vec(vector_size=20,window=5,min_count=1,workers=2,dm=1,epochs=20)
model_sent2.build_vocab(tagged_sent2)

model_sent3 = Doc2Vec(vector_size=20,window=5,min_count=1,workers=2,dm=1,epochs=20)
model_sent3.build_vocab(tagged_sent3)


In [43]:
for epoch in range(20):
    model_sent.train(tagged_sent,epochs=20,total_examples=model_sent.corpus_count)
    model_sent1.train(tagged_sent1,epochs=20,total_examples=model_sent1.corpus_count)
    model_sent2.train(tagged_sent2,epochs=20,total_examples=model_sent2.corpus_count)
    model_sent3.train(tagged_sent3,epochs=20,total_examples=model_sent3.corpus_count)   
    # print("Epoch #{} is complete.".format(epoch+1))
print("Done")

Done


In [44]:
doc_sent_sent1=[cos_sim(model_sent.docvecs[i],model_sent1.docvecs[i]) for i in range(len(model_sent.docvecs[0]))]
doc_sent_sent2=[cos_sim(model_sent.docvecs[i],model_sent2.docvecs[i]) for i in range(len(model_sent.docvecs[0]))]
doc_sent_sent3=[cos_sim(model_sent.docvecs[i],model_sent3.docvecs[i]) for i in range(len(model_sent.docvecs[0]))]

print(np.mean(doc_sent_sent1),np.mean(doc_sent_sent2),np.mean(doc_sent_sent3))


0.71514404 0.7144384 0.6080502


In [45]:
with open("jumbled_results.txt","a") as f:
  f.write("Doc2Vec \n")
  f.write("n=1 %f" % np.mean(doc_sent_sent1))
  f.write("\n")
  f.write("n=2 %f" % np.mean(doc_sent_sent2))
  f.write("\n")
  f.write("n=3 %f" % np.mean(doc_sent_sent3))
  f.write("\n")
  f.write("\n")

# InferSent

In [46]:
# import stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint

import numpy as np
import torch

In [47]:
!mkdir GloVe
!curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip GloVe/glove.840B.300d.zip -d GloVe/
#!mkdir fastText
#!curl -Lo fastText/crawl-300d-2M.vec.zip https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
#!unzip fastText/crawl-300d-2M.vec.zip -d fastText/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   315    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   352    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2075M  100 2075M    0     0  5162k      0  0:06:51  0:06:51 --:--:-- 5433k
Archive:  GloVe/glove.840B.300d.zip
  inflating: GloVe/glove.840B.300d.txt  


In [48]:
!mkdir encoder
!curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
!curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  30.0M      0  0:00:04  0:00:04 --:--:-- 32.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  34.9M      0  0:00:04  0:00:04 --:--:-- 34.9M


In [49]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [50]:
# Load model
from models import InferSent
model_version = 1
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [51]:
# Keep it on CPU or put it on GPU
use_cuda = False
model = model.cuda() if use_cuda else model

In [52]:
# If infersent1 -> use GloVe embeddings. If infersent2 -> use InferSent embeddings.
W2V_PATH = 'GloVe/glove.840B.300d.txt' if model_version == 1 else 'fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

In [53]:
# Load embeddings of K most frequent words
model.build_vocab_k_words(K=100000)

Vocab size : 100000


In [54]:
infer_embed_sent=model.encode(sent,tokenize = True)
infer_embed_sent1=model.encode(sent1,tokenize = True)
infer_embed_sent2=model.encode(sent2,tokenize = True)
infer_embed_sent3=model.encode(sent3,tokenize = True)



/content/models.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentences = np.array(sentences)[idx_sort]


In [55]:
infer_sent_sent1=[cos_sim(infer_embed_sent[i],infer_embed_sent1[i]) for i in range(len(infer_embed_sent))]
infer_sent_sent2=[cos_sim(infer_embed_sent[i],infer_embed_sent2[i]) for i in range(len(infer_embed_sent))]
infer_sent_sent3=[cos_sim(infer_embed_sent[i],infer_embed_sent3[i]) for i in range(len(infer_embed_sent))]



In [56]:
print(np.mean(infer_sent_sent1),np.mean(infer_sent_sent2),np.mean(infer_sent_sent3))

0.9778838 0.96324265 0.9531452


In [57]:
with open("jumbled_results.txt","a") as f:
  f.write("InferSent \n")
  f.write("n=1 %f" % np.mean(infer_sent_sent1))
  f.write("\n")
  f.write("n=2 %f" % np.mean(infer_sent_sent2))
  f.write("\n")
  f.write("n=3 %f" % np.mean(infer_sent_sent3))
  f.write("\n")
  f.write("\n")